In [7]:
import nltk
import pandas as pd 
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string 

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv('SMSSpamCollection.tsv', sep='\t')
data.columns = ['label', 'body_text']

def count_punch(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text)-text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punch%'] = data['body_text'].apply(lambda x: count_punch(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)

X_tfidf = tfidf_vect.fit_transform(data['body_text'])

X_features = pd.concat([data['body_len'], data['punch%'], pd.DataFrame(X_tfidf.toarray())], axis = 1)
X_features.head()

,body_len,punch%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Exploring GradientBoostingclassifier Attributes and hyperparameters

In [8]:
from sklearn.ensemble import GradientBoostingClassifier

In [13]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [14]:
X_train , x_test, Y_train, y_test = train_test_split(X_features, data['label'], test_size = 0.2)

In [15]:
def train_GB(est, max_depth, lr):
    gb = GradientBoostingClassifier(n_estimators=est, max_depth=max_depth, learning_rate=lr)
    gb_model = gb.fit(X_train, Y_train)
    y_pred =gb_model.predict(x_test) 
    precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')
    print('Est: {} / Depth :{}  / lr : {} --------- precision : {} / Accuracy : {}'. format(est, max_depth , lr, round(precision, 3), round(recall, 3), round((y_pred == y_test).sum()/len(y_pred), 3)))

In [ ]:
for n_est in [50 , 100, 150]:
    for max_depth in [3, 7, 11, 15]:
        for lr in [0.01 , 0.1, 1]:
            train_GB(n_est, max_depth, lr)

c:\users\manish\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Est: 50 / Depth :3  / lr : 0.01 --------- precision : 0.0 / Accuracy : 0.0
Est: 50 / Depth :3  / lr : 0.1 --------- precision : 0.897 / Accuracy : 0.689
Est: 50 / Depth :3  / lr : 1 --------- precision : 0.873 / Accuracy : 0.756
Est: 50 / Depth :7  / lr : 0.01 --------- precision : 1.0 / Accuracy : 0.012
Est: 50 / Depth :7  / lr : 0.1 --------- precision : 0.887 / Accuracy : 0.768
Est: 50 / Depth :7  / lr : 1 --------- precision : 0.874 / Accuracy : 0.805


c:\users\manish\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Est: 50 / Depth :11  / lr : 0.01 --------- precision : 0.0 / Accuracy : 0.0
Est: 50 / Depth :11  / lr : 0.1 --------- precision : 0.88 / Accuracy : 0.805
Est: 50 / Depth :11  / lr : 1 --------- precision : 0.878 / Accuracy : 0.835


c:\users\manish\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Est: 50 / Depth :15  / lr : 0.01 --------- precision : 0.0 / Accuracy : 0.0
Est: 50 / Depth :15  / lr : 0.1 --------- precision : 0.879 / Accuracy : 0.799
Est: 50 / Depth :15  / lr : 1 --------- precision : 0.897 / Accuracy : 0.854
Est: 100 / Depth :3  / lr : 0.01 --------- precision : 0.874 / Accuracy : 0.463
Est: 100 / Depth :3  / lr : 0.1 --------- precision : 0.903 / Accuracy : 0.738
Est: 100 / Depth :3  / lr : 1 --------- precision : 0.884 / Accuracy : 0.787
Est: 100 / Depth :7  / lr : 0.01 --------- precision : 0.917 / Accuracy : 0.61
Est: 100 / Depth :7  / lr : 0.1 --------- precision : 0.89 / Accuracy : 0.793
Est: 100 / Depth :7  / lr : 1 --------- precision : 0.885 / Accuracy : 0.799
Est: 100 / Depth :11  / lr : 0.01 --------- precision : 0.895 / Accuracy : 0.677
Est: 100 / Depth :11  / lr : 0.1 --------- precision : 0.875 / Accuracy : 0.811
Est: 100 / Depth :11  / lr : 1 --------- precision : 0.895 / Accuracy : 0.829
Est: 100 / Depth :15  / lr : 0.01 --------- precision : 0.8